In [1]:
from os import path
xml_path = path.relpath("Data/pages_8_47.xml")

from bs4 import BeautifulSoup
with open(xml_path, encoding='utf8') as file:
    contents = file.read()
    soup = BeautifulSoup(contents, 'xml')

In [2]:
import re

In [3]:
#soup.find_all('hi', string=re.compile(r'Sonntag'))

Trying to get all the text - and placeTags for Sunday:

In [3]:
start_tag = 'hi'
start_string = 'Sonntag'
end_tag = 'hi'
end_string = 'Montag.'
place_tag = 'placeName'
should_start_parsing = False

In [4]:
Sonntag = []
Places = []
for line in soup.find_all('l'):
    if line.string == start_string and line.string.parent.name == start_tag:
        should_start_parsing = True  

    if should_start_parsing:
        text = line.get_text()
        Sonntag.append(text)
        for child in line.descendants:
            if child.name == place_tag:
                for text in child.strings:
                    #hier gibts Verbesserungsbedarf! -> Ziel ist, das die strings von placeTags, die durch andere Tags unterbrochen werden, 
                    #z.B- <placeName> Schloss <persName> Esterhazy </persName> </placeName>
                    #nicht als getrennte, sondern ein string gespeichert werden. 
                    Places.append(text)          

    if line.string == end_string and line.string.parent.name == end_tag:
        should_start_parsing = False  
        break

In [6]:
#print(Sonntag)
#print(Places)

Cleaning and structuring the list of places:

In [5]:
while(" " in Places):
    Places.remove(" ")
print(Places) 

['k. k. Burg', 'Burgplatz', 'Burgthor', 'kaiserlichen Stallungen', 'Ge=', 'bäude der ungarischen Garde', 'Palais des ', 'Fürsten Auersperg', 'Gebäude des ', 'Geographi=', 'schen (Militär=)Jnstituts', 'Kriminal=', 'Gebäude', 'Kahlengebirge', 'Mariahilfer Hauptstraße', 'Schön=', 'brunn', 'Grena=', 'dierkaserne', 'Getreide=', 'markt', 'artesischen Brunnen', 'Akazien=Alleen', 'Wien', 'Kettenstege', 'der steinernen Brücke', 'Polytechnische Jnstitut', 'Karlskirche', 'Burg', 'Ritter=', 'Ceremoniensaal', 'Appartements ', 'Sr Majestät des Kaisers', 'Rittersaal', 'Antiken=', 'Mineralienkabinet', 'astronomische Thurm', 'Burg', 'Hofbiblio=', 'thek=Gebäudes', 'Josephsplatze', 'Palais des ', 'Erzherzogs Karl', 'Burg=', 'platze', 'Volksgartens', 'Burg=', 'gartens', 'Burggarten', 'Gewächshäuser', 'Wall', 'Palais des ', 'Erzherzogs Karl', 'Spitalplatze', 'Hofbau=', 'amte', 'Kärnthnerthor=Theater', 'Wall', 'Kärnthnerthore', 'Kärnthnerstraße', 'Wieden', 'Baden', 'Starhembergische ', 'Freihaus', 'Polytech

In [6]:
clean_places = []
for e in Places:
    if clean_places and clean_places[-1].endswith('='):
        clean_places[-1] = clean_places[-1][:-1] + e
    elif clean_places and (clean_places[-1].endswith('des ') or clean_places[-1].endswith('des')):
        clean_places[-1] = clean_places[-1] + ' ' + e
    elif clean_places and (clean_places[-1].endswith('der ') or clean_places[-1].endswith('der')): 
        clean_places[-1] = clean_places[-1] + ' ' + e     
    elif clean_places and clean_places[-1].endswith('den '): 
        clean_places[-1] = clean_places[-1] + ' ' + e 
    elif clean_places and (clean_places[-1].endswith('Fürsten ') or clean_places[-1].endswith('.')):
        clean_places[-1] = clean_places[-1] + ' ' + e         
    else:
        clean_places.append(e)


In [7]:
print(clean_places)

['k. k. Burg', 'Burgplatz', 'Burgthor', 'kaiserlichen Stallungen', 'Gebäude der ungarischen Garde', 'Palais des  Fürsten Auersperg', 'Gebäude des  Geographischen (Militär=)Jnstituts', 'KriminalGebäude', 'Kahlengebirge', 'Mariahilfer Hauptstraße', 'Schönbrunn', 'Grenadierkaserne', 'Getreidemarkt', 'artesischen Brunnen', 'Akazien=Alleen', 'Wien', 'Kettenstege', 'der steinernen Brücke', 'Polytechnische Jnstitut', 'Karlskirche', 'Burg', 'RitterCeremoniensaal', 'Appartements ', 'Sr Majestät des Kaisers', 'Rittersaal', 'AntikenMineralienkabinet', 'astronomische Thurm', 'Burg', 'Hofbibliothek=Gebäudes Josephsplatze', 'Palais des  Erzherzogs Karl', 'Burgplatze', 'Volksgartens', 'Burggartens', 'Burggarten', 'Gewächshäuser', 'Wall', 'Palais des  Erzherzogs Karl', 'Spitalplatze', 'Hofbauamte', 'Kärnthnerthor=Theater', 'Wall', 'Kärnthnerthore', 'Kärnthnerstraße', 'Wieden', 'Baden', 'Starhembergische ', 'Freihaus', 'Polytechnische Jnstitut', 'Karlskirche', 'Starhembergische Frelhaus', 'Wienerwaldes

Some manual refinements, necessary (hopefully) only in this case: 

In [8]:
import functools

In [9]:
clean_places.remove('ist Wien = Donau?')
clean_places.remove('Vorstadt = Bezirk')
clean_places.index('Wienerwaldes Schneeberg')

50

put together: große Steinbrücke, Starhembergisches Freihaus, Appartements Sr. Majestät, 'fürstlich''Esterhazische sogenannte ''rothe Haus'
seperated: Hofbibliotheksgebäude - Josephplatz; Wienerwaldes - Schneeberg; 

In [10]:
clean_places[73:75] = [functools.reduce(lambda x, y: x + ' ' + y, clean_places[73:75])]
print(clean_places[73])

große Steinbrücke


In [11]:
clean_places[45:47] = [functools.reduce(lambda x, y: x + y, clean_places[45:47])]
print(clean_places[45])

Starhembergische Freihaus


In [12]:
clean_places[22:24] = [functools.reduce(lambda x, y: x + y, clean_places[22:24])]
print(clean_places[22])

Appartements Sr Majestät des Kaisers


In [13]:
clean_places[154:157] = [functools.reduce(lambda x, y: x + y, clean_places[154:157])]
print(clean_places[154])

fürstlichEsterhazische sogenannte rothe Haus


In [14]:
clean_places.insert(27, 'Hofbibliotek=Gebäudes')
clean_places.insert(28, 'Josephsplatze')
clean_places.remove('Hofbibliothek=Gebäudes Josephsplatze')

In [15]:
clean_places.insert(49, 'Wienerwaldes')
clean_places.insert(50, 'Schneeberg')
clean_places.remove('Wienerwaldes Schneeberg')

In [16]:
clean_places[85:87] = [functools.reduce(lambda x, y: x + ' ' + y, clean_places[85:87])]
print(clean_places[85])

Kloster und Kirche der  Dominikaner


In [17]:
clean_places_text = open('Data/clean_places_text.txt','w', encoding='utf8')
for item in clean_places:
	clean_places_text.write(item+"\n")
clean_places_text.close()

There are several possibilities to normalize text - one idea could be, using the spacy-package, to lemmatize the places - the problem here although is that spacy splits words that belong together - as names - and the results of lemmatization themselves do not look too good ("Stallunge")

In [20]:
#import spacy
#nlp = spacy.load("de_core_news_sm")

In [21]:
#places_text = ' '.join(clean_places)
#nlp_places = nlp(places_text)

In [22]:
#lemma_list = []
#for word in nlp_places: 
    #lemma = word.lemma_
    #lemma_list.append(lemma)    
#print(lemma_list)

Another Option would be fuzzy string matching 

In [18]:
import difflib as dl

In [19]:
Similiar_words = []
for word in clean_places:
    close_words = dl.get_close_matches(word, clean_places, cutoff= 0.9)
    Similiar_words.append(close_words)

In [20]:
place_list = []
for sublist in Similiar_words:
    sublist = sorted(sublist)
    place = sublist[0]
    place_list.append(place)

In [21]:
print(len(place_list))
print(place_list)
place_list_text= open('Data/place_list_text.txt','w', encoding='utf8')
for item in place_list:
	place_list_text.write(item+"\n")
place_list_text.close()


188
['k. k. Burg', 'Burgplatz', 'Burgthor', 'kaiserlichen Stallungen', 'Gebäude der ungarischen Garde', 'Palais des  Fürsten Auersperg', 'Gebäude des  Geographischen (Militär=)Jnstituts', 'KriminalGebäude', 'Kahlengebirge', 'Mariahilfer Hauptstraße', 'Schönbrunn', 'Grenadierkaserne', 'Getreidemarkt', 'artesischen Brunnen', 'Akazien=Alleen', 'Wien', 'Kettenstege', 'der steinernen Brücke', 'Polytechnische Jnstitut', 'Karlskirche', 'Burg', 'RitterCeremoniensaal', 'Appartements Sr Majestät des Kaisers', 'Rittersaal', 'AntikenMineralienkabinet', 'astronomische Thurm', 'Burg', 'Hofbibliotek=Gebäudes', 'Josephsplatze', 'Palais des  Erzherzogs Karl', 'Burgplatz', 'Volksgarten', 'Burggarten', 'Burggarten', 'Gewächshäuser', 'Wall', 'Palais des  Erzherzogs Karl', 'Spitalplatze', 'Hofbauamte', 'Kärnthnerthor=Theater', 'Wall', 'Kärnthnerthore', 'Kärnthnerstraße', 'Wieden', 'Baden', 'Starhembergische Freihaus', 'Polytechnische Jnstitut', 'Karlskirche', 'Starhembergische Freihaus', 'Wienerwaldes', 'S

It worked pretty well - but, we still have some cases like "Sommerpalais des Fürsten Schwarzenberg" and "Schwarzenbergisches Palais", which are the same, but have not been matched...

Next is to create a dataframe containing the coordinates for each place - i'm trying to use the df from the Wien Geschichte Wiki (https://www.data.gv.at/katalog/dataset/wien-geschichte-wiki#resources) containing gis-data for historic places and sights in vienna

In [22]:
import pandas as pd
csv_path = path.relpath("Data/KULTURWIKIOGD.csv")
usecolumns = ['OBJECTID', 'SHAPE', 'ADRESSE', 'SEITENNAME']
coordinates = pd.read_csv(csv_path,  index_col="OBJECTID", usecols=usecolumns)

In [28]:
coordinates.head()

,SHAPE,ADRESSE,SEITENNAME
OBJECTID,,,
294006,POINT (16.24449608644173 48.20765995647323),"14., Mondweg 91",Franz-Sauer-Park
294007,POINT (16.258093352548578 48.19709893851195),"13., Rußpekgasse 3",Franz-Schimon-Park
294008,POINT (16.26067781801285 48.182308544859616),"13., Schweizertalstraße 29D",Franz-Schmidt-Park
294009,POINT (16.338812997907947 48.188874975260035),"15., U-Bahn-Bogen 1",Franz-Schwarz-Park
294010,POINT (16.39077256132719 48.22313059046534),"02., Nordbahnstraße 49",Franziska-Löw-Park


In [23]:
from thefuzz import process, fuzz 

In [24]:
coordinates['MATCH_SCORE'] = coordinates['SEITENNAME'].apply(lambda x: process.extractOne(x, place_list, scorer=fuzz.ratio)[1])

In [25]:
coordinates = coordinates.sort_values('MATCH_SCORE', ascending=False)

In [26]:
coordinates_choice = coordinates.drop(coordinates[coordinates['MATCH_SCORE'] <= 84].index)

In [27]:
coordinates_choice['NEUE_NAMEN'] = coordinates_choice['SEITENNAME'].apply(lambda x: process.extractOne(x, place_list, scorer=fuzz.ratio)[0])

In [28]:
coordinates_choice

,SHAPE,ADRESSE,SEITENNAME,MATCH_SCORE,NEUE_NAMEN
OBJECTID,,,,,
294267,POINT (16.392549994163186 48.19242504251556),"03., Steingasse 38",Landstraße,100,Landstraße
285197,POINT (16.37032521081372 48.21879800173373),"02., Augartenbrücke 201",Augartenbrücke,100,Augartenbrücke
294201,POINT (16.33306156333985 48.27620752069916),"19., Villenweg 58",Kahlenberg,100,Kahlenberg
281691,POINT (16.380340127652488 48.19424811295102),"03., Prinz-Eugen-Straße 27 Obj. 8",Belvedere,100,Belvedere
281459,POINT (16.37642189268371 48.213588305837256),"02., Obere Donaustraße 93",Dianabad,100,Dianabad
...,...,...,...,...,...
283197,POINT (16.370205880226816 48.20390709779573),"01., Philharmonikerstraße 2",Kärntner Straße 38,85,Kärnthnerstraße
294500,POINT (16.374430808795385 48.21228542778926),"01., Morzinplatz 1",Ruprechtsstiege,85,St. Ruprechtstiege
283196,POINT (16.371037818771857 48.204879820114485),"01., Kärntner Straße 37",Kärntner Straße 37,85,Kärnthnerstraße


Das Problem ist ja, dass ein Ort wie Ferdinands-Brücke im Prinzip dasselbe ist wie Ferdinandsbrücke, aber im Vergleich einen niederigen Score hat...

In [55]:
coordinates_choice = coordinates_choice.drop_duplicates(subset=['NEUE_NAMEN'])

In [56]:
coordinates_choice

,SHAPE,ADRESSE,SEITENNAME,MATCH_SCORE,NEUE_NAMEN
OBJECTID,,,,,
294267,POINT (16.392549994163186 48.19242504251556),"03., Steingasse 38",Landstraße,100,Landstraße
285197,POINT (16.37032521081372 48.21879800173373),"02., Augartenbrücke 201",Augartenbrücke,100,Augartenbrücke
294201,POINT (16.33306156333985 48.27620752069916),"19., Villenweg 58",Kahlenberg,100,Kahlenberg
281691,POINT (16.380340127652488 48.19424811295102),"03., Prinz-Eugen-Straße 27 Obj. 8",Belvedere,100,Belvedere
281459,POINT (16.37642189268371 48.213588305837256),"02., Obere Donaustraße 93",Dianabad,100,Dianabad
294202,POINT (16.33306156333985 48.27620752069916),"19., Am Kahlenberg K Stephanie-Warte",Kahlengebirge,100,Kahlengebirge
281656,POINT (16.371945115880457 48.207757987582276),"01., Singerstraße 2",Bastei,100,. Bastei
285722,POINT (16.37186617386608 48.19824509903096),"04., Karlsplatz 10(KIRCHE)",Karlskirche,100,Karlskirche
294284,POINT (16.423993251728795 48.19766541152966),"02., Heustadlsteg 213",Leopoldstadt,100,Leopoldstadt


In [57]:
place_coord  = coordinates_choice.drop(columns = ['SEITENNAME', 'MATCH_SCORE'])


In [58]:
place_coord = place_coord.set_index('NEUE_NAMEN')

In [59]:
place_coord = place_coord.reindex(index=place_list)
place_coord = place_coord.reset_index()

In [62]:
place_coord.reset_index(inplace=True)

In [63]:
place_coord

,index,NEUE_NAMEN,SHAPE,ADRESSE
0,0,k. k. Burg,NaN,NaN
1,1,Burgplatz,NaN,NaN
2,2,Burgthor,NaN,NaN
3,3,kaiserlichen Stallungen,NaN,NaN
4,4,Gebäude der ungarischen Garde,NaN,NaN
...,...,...,...,...
183,183,Volksgarten,POINT (16.36175523794077 48.20796405847014),"01., Volksgarten THESEUSTEMPEL"
184,184,Theseustempel,POINT (16.36168623807803 48.208383057004575),"01., Volksgarten THESEUSTEMPEL"
185,185,Hof=Burgwache,NaN,NaN
186,186,Schweizerhof,POINT (16.36584321538764 48.20687606097441),"01., Hofburg B Schweizertrakt"


In [64]:
place_coord.to_csv('Data/Sonntag_coord.csv', sep=',', index=False, encoding='utf-8')

Nächster Versuch mit dem Gazeteer Steinhausen Plan: 

In [66]:
xlsx_path = path.relpath("Data/Gazetteer_Steinhausenplan_V5.xlsx")
usecolumns = ['ID_neu', 'Toponym', 'Sicherheit', 'Longitude', 'Latitude']
Steinhausen_coordinates = pd.read_excel(xlsx_path, usecols=usecolumns, index_col=0)  

In [67]:
Steinhausen_coordinates.head()

,Toponym,Sicherheit,Longitude,Latitude
ID_neu,,,,
1,3 Cron:,hoch,16.376307,48.211565
2,3 Fisch,hoch,16.374240,48.210826
3,3 Fisch.,hoch,16.374100,48.210932
4,3 Rueben,mittel,16.370820,48.208375
5,5. Cronen,hoch,16.368902,48.211668


In [68]:
Steinhausen_coordinates['MATCH_SCORE'] = Steinhausen_coordinates['Toponym'].apply(lambda x: process.extractOne(x, place_list, scorer=fuzz.ratio)[1])
Steinhausen_coordinates['NEUE_NAMEN'] = Steinhausen_coordinates['Toponym'].apply(lambda x: process.extractOne(x, place_list, scorer=fuzz.ratio)[0])
Steinhausen_coordinates = Steinhausen_coordinates.sort_values('MATCH_SCORE', ascending=False)

Applied processor reduces input query to empty string, all comparisons will have score 0. [Query: '<...>']
Applied processor reduces input query to empty string, all comparisons will have score 0. [Query: '<...>']
Applied processor reduces input query to empty string, all comparisons will have score 0. [Query: '<...>']
Applied processor reduces input query to empty string, all comparisons will have score 0. [Query: '<...>']
Applied processor reduces input query to empty string, all comparisons will have score 0. [Query: '<...>']
Applied processor reduces input query to empty string, all comparisons will have score 0. [Query: '<...>']
Applied processor reduces input query to empty string, all comparisons will have score 0. [Query: '<...>']
Applied processor reduces input query to empty string, all comparisons will have score 0. [Query: '<...>']
Applied processor reduces input query to empty string, all comparisons will have score 0. [Query: '<...>']
Applied processor reduces input query

In [70]:
Steinhausen_coordinates.head(50)

,Toponym,Sicherheit,Longitude,Latitude,MATCH_SCORE,NEUE_NAMEN
ID_neu,,,,,,
1213,Stuebenthor.,hoch,16.378590,48.207739,95,Stubenthor
1212,Stuben Thor,hoch,16.380630,48.207246,95,Stubenthor
165,Burg Thor,hoch,16.363591,48.207307,94,Burgthor
819,LEO POLD STADT,hoch,16.378814,48.212619,92,Leopoldstadt
1083,Schotten Thor,hoch,16.363595,48.212821,92,Schottenthore
648,Herrn Gassen,hoch,16.365492,48.210009,91,Herrngasse
1079,Schön Brun,hoch,16.370375,48.210180,90,Schönbrunn
988,Renn=Wege.,mittel,16.379695,48.197128,88,Rennweg
1249,unter den Weisgarbern,hoch,16.388423,48.212241,88,unter den Weißgärbern


In [71]:
Steinhausen_coordinates = Steinhausen_coordinates.drop(Steinhausen_coordinates[Steinhausen_coordinates['MATCH_SCORE'] <= 80].index)

Das Problem beim Steinhausenplan ist, dass da einige Orts-Schreibweise doppelt vorkommen, z.B. Stuebenthore und Stuben Thor, die offensichtlich dasselbe sind, aber unterschiedliche koordinaten haben. Für meine Methode darf's aber nur einen Stefansplatz geben, sonst funktioniert später die indexierung nicht. 
Darum probier ichs jetzt mal so, & wenn das dann zu ungenau ist, stell ichs nochmal um. 

In [73]:
Steinhausen_choice = Steinhausen_coordinates.drop_duplicates(subset=['NEUE_NAMEN'])

In [74]:
Steinhausen_choice = Steinhausen_choice.set_index('NEUE_NAMEN')
Steinhausen_choice = Steinhausen_choice.reindex(index=place_list)
Steinhausen_choice = Steinhausen_choice.reset_index()

In [75]:
Steinhausen_choice.reset_index(inplace=True)

In [76]:
Steinhausen_choice

,index,NEUE_NAMEN,Toponym,Sicherheit,Longitude,Latitude,MATCH_SCORE
0,0,k. k. Burg,NaN,NaN,NaN,NaN,NaN
1,1,Burgplatz,NaN,NaN,NaN,NaN,NaN
2,2,Burgthor,Burg Thor,hoch,16.363591,48.207307,94.0
3,3,kaiserlichen Stallungen,NaN,NaN,NaN,NaN,NaN
4,4,Gebäude der ungarischen Garde,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
183,183,Volksgarten,NaN,NaN,NaN,NaN,NaN
184,184,Theseustempel,NaN,NaN,NaN,NaN,NaN
185,185,Hof=Burgwache,NaN,NaN,NaN,NaN,NaN
186,186,Schweizerhof,NaN,NaN,NaN,NaN,NaN


In [77]:
Steinhausen_choice.to_csv('Data/Steinhausen_Sonntag_coord.csv', sep=',', index=False, encoding='utf-8')